# Webscrape Dataclean

In [20]:
import pandas as pd
import numpy as np
import nltk
import time
import re
import string


nltk.download("words")
from nltk.corpus import words
pd.set_option('display.max_columns', 10000)

[nltk_data] Error loading words: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:1108)>


In [2]:
google_df=pd.read_csv('google_data.csv',error_bad_lines=False)

b'Skipping line 4715: expected 36 fields, saw 37\nSkipping line 9582: expected 36 fields, saw 37\n'


In [3]:
google_df.index=list(google_df.index)
google_df=google_df[google_df["language"]=='en']
google_df=pd.DataFrame(google_df.tweet, columns=['tweet','company'])
google_df.index=list(google_df.index)

In [4]:
punctuation=[",",".","#","!","@","$","?","$","%","&","-","_",":","=", "/",";",'(',")", "{","}","|"]
apple=['iPhone',"iphones",'i phones',"iPad&quot;","iphone ",'i Pad','i Pad',"Apple.","iPad?","#iPad","#iPhone",'iPad','Mac',"iphone,",'iPod','ipod','ipad','mac','Apple','apple','Apple\'s','apple\'s',"#Apple",'#apple','I-Pad']
google=['Google','google','android','Android','Google.','android,',"#android","Google's",'google\'s','googles' ,'#google','#Google']

In [5]:
def company_extract(df):
    for index in df.index:
        tweet="".join(i if i not in punctuation else " " for i in df.tweet[index])
        condition=0
        df.tweet[index]=tweet


        if bool(set(tweet.split())&set(apple))==True:
            df.company[index]=(list(set(tweet.split())&set(apple))[0].title().strip("'s")).strip('\'S')
            condition=1
        else:
            condition=0

        if (bool(set(tweet.split())&set(google))==True) and condition==0:
            df.company[index]=(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')

        elif (bool(set(tweet.split())&set(google))==True) and condition==1:
            df.company[index]+= " & "+(list(set(tweet.split())&set(google))[0].title().strip('\'s')).strip('\'S')

In [6]:

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('[%s]' % re.escape(string.punctuation), ' ', text)
    text = re.sub('\w*\d\w*', ' ', text)
    text = re.sub(r'^https?:\/\/.*[\r\n]*', ' ', text, flags=re.MULTILINE)

    return text

round1 = lambda x: clean_text_round1(x)

In [7]:
company_extract(google_df)

In [8]:
google_df['clean_tweet']=google_df.tweet.apply(round1)

In [9]:
apple_df=pd.read_csv('apple_data.csv',error_bad_lines=False)

b'Skipping line 828: expected 36 fields, saw 55\nSkipping line 1263: expected 36 fields, saw 49\nSkipping line 1283: expected 36 fields, saw 49\nSkipping line 1294: expected 36 fields, saw 49\nSkipping line 1306: expected 36 fields, saw 49\nSkipping line 1370: expected 36 fields, saw 52\nSkipping line 2062: expected 36 fields, saw 46\nSkipping line 2065: expected 36 fields, saw 49\nSkipping line 3352: expected 36 fields, saw 37\n'


In [10]:
apple_df.index=list(apple_df.index)
apple_df=apple_df[apple_df["language"]=='en']
apple_df=pd.DataFrame(apple_df.tweet, columns=['tweet','company'])
apple_df.index=list(apple_df.index)

In [12]:
company_extract(apple_df)

In [13]:
apple_df.company.value_counts()

Apple               5710
Iphone               132
Apple & Google       122
Apple & Android       63
Ipad                  43
Mac                   31
Google                 7
Iphone & Android       7
Iphone & Google        2
Android                2
Mac & Google           1
Ipod                   1
Ipad & Android         1
Name: company, dtype: int64

In [14]:
google_df.company.value_counts()

Google              5312
Apple & Google       107
Android               33
Iphone & Google       11
Ipad & Google          5
Mac & Google           4
Apple & Android        3
Iphone & Android       2
Name: company, dtype: int64

In [23]:
def tweet_to_dic(df):
    dictionary={}
    companies=list(df.company.unique())
    iteration=1
    for company in companies:
        print("This is iteration: "+str(iteration)+" of "+str(len(companies)))
        tweet=[]
        for index in df[df["company"]==company].index:
            print('\r' +str(index), end="", flush=True)
            tweet.append(" ".join(i if i  in words.words() else "" for i in df.tweet[index].lower().split()))
        dictionary[company]=tweet
        print("\n")
        iteration+=1
    return dictionary

In [25]:
g_comp_dic=tweet_to_dic(google_df)

This is iteration: 1 of 9
10092

This is iteration: 2 of 9
10052

This is iteration: 3 of 9
10025

This is iteration: 4 of 9


This is iteration: 5 of 9
6984

This is iteration: 6 of 9
9582

This is iteration: 7 of 9
8250

This is iteration: 8 of 9
6514

This is iteration: 9 of 9
6524



In [26]:
a_comp_dic=tweet_to_dic(apple_df)

This is iteration: 1 of 14
10073

This is iteration: 2 of 14


This is iteration: 3 of 14
9930

This is iteration: 4 of 14
9806

This is iteration: 5 of 14
3622

This is iteration: 6 of 14
10070

This is iteration: 7 of 14
10014

This is iteration: 8 of 14
10064

This is iteration: 9 of 14
9289

This is iteration: 10 of 14
8333

This is iteration: 11 of 14
3329

This is iteration: 12 of 14
4463

This is iteration: 13 of 14
5901

This is iteration: 14 of 14
6625



In [27]:
dfg=pd.DataFrame(columns=['tweets','company'])
dfg['tweets']=list(g_comp_dic.values())
dfg['company']=list(g_comp_dic.keys())

In [28]:
dfa=pd.DataFrame(columns=['tweets','company'])
dfa['tweets']=list(a_comp_dic.values())
dfa['company']=list(a_comp_dic.keys())

In [29]:
dfg.to_csv('dfg.csv')
dfa.to_csv('dfa.csv')